# Advent of Code 2020 - Day 10

In [33]:
import java.io.File

val adapters: List<Int> = File("Day10.input.txt")
    .bufferedReader()
    .readLines()
    .map(String::toInt)

## Part 1

Find a chain that uses all of the `adapters` to connect the outlet and the device. Each adapter is listed with rating and connect to a source 1-3 jolts lower than the rating. The device is always 3 higher than the highest adapter. What is the number of 1-jolt differences multiplied by the number of 3-jolt differences?

In [34]:
(adapters + 0 + (adapters.maxOrNull()!! + 3))
    .sorted()
    .windowed(size = 2)
    .groupingBy { (lhs, rhs) -> rhs - lhs }
    .eachCount()
    .let { it[1]!! * it[3]!! }

1890

### Notes

This one was really straightforward thanks to some handy Kotlin functions. The input guarantees to us that all adapters need to connect, so we can just sort the list. Next we just have to count the differences between each adapter, which we can achieve by windowing each adapter and then counting the 3 and 1 differences.

## Part 2

With the previous rules from part 1 still applying, find every arrangement of adapters that connect the outlet to the device. Count the total number of distinct ways to configure the outlets.

In [35]:
@JvmInline
value class Adapter(val range: IntRange)

fun Adapter.overlaps(other: Adapter): Boolean = 
    (range.start <= other.range.start &&
    range.start <= other.range.endInclusive && 
    range.endInclusive >= other.range.start)

fun Adapter.connections(adapters: List<Adapter>): List<Adapter> = adapters.filter { overlaps(it) }

fun countConfigs(from: Adapter, to: List<Adapter>): Long {
    val connections = from.connections(to)
    if (connections.size == 0) return 1L

    return connections.sumOf { adapter -> memoized(adapter, to - adapter) }
}

val memos: MutableMap<Adapter, Long> = mutableMapOf()

fun ((Adapter, List<Adapter>) -> Long).memoize(): ((Adapter, List<Adapter>) -> Long) = 
    { adapter, adapters -> memos.getOrPut(adapter) { this(adapter, adapters) } }

val memoized = ::countConfigs.memoize()

(adapters + (adapters.maxOrNull()!! + 3))
    .map { Adapter((it-3)..it) }
    .sortedBy { it.range.endInclusive }
    .let {
        memoized(Adapter(-3..0), it)
    }

49607173328384

### Notes

This idea can be modeled as a depth-first search. At each adapter find the list of valid connections and then find all of their valid connections and then count the routes that lead to the end. Basically every valid connection is a "branch" and only the ones that connect to the end are counted. However, the input is really large, so calculating each route will take a really long time. We can "memoize" the call by storing the result of each adapter that was visited. If a subsequent visit to that adapter occurs, rather than calculating the configurations again, we can just return what was calculated previously. We know the configuration is unique because the adapter was not previously stored.